# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df=pd.read_csv("data.csv")
df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Cayenne,88,GF,1585841405,94,4.93,-52.33,26.00,6.20
1,Jamestown,90,US,1585841524,48,42.10,-79.24,7.00,6.70
2,Vila Velha,40,BR,1585841524,66,-20.33,-40.29,30.56,3.60
3,Codrington,100,AU,1585841524,84,-38.27,141.97,14.39,4.11
4,Lebu,0,CL,1585841524,48,-37.62,-73.65,17.44,7.80
...,...,...,...,...,...,...,...,...,...
553,Pisco,36,PE,1585842206,30,-13.70,-76.22,25.56,2.60
554,Izhevskoye,100,RU,1585842206,71,54.56,40.87,3.87,4.49
555,Yaan,49,NG,1585842206,39,7.38,8.57,35.79,3.15
556,Magdagachi,100,RU,1585842206,90,53.45,125.80,-9.34,2.38


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [185]:
location=df[["Lat","Lng"]]
weight=df["Humidity"]


In [196]:
fig=gmaps.figure()

heat_layer = gmaps.heatmap_layer(location, weights=weight, 
                                 dissipating=False, max_intensity=100, point_radius=1)
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [113]:
vacation_df=df.loc[(df["Max Temp"]<85) & (df["Cloudiness"]<10) & (df["Wind Speed"]<20)]

vacation_df.dropna()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
4,Lebu,0,CL,1585841524,48,-37.62,-73.65,17.44,7.80
5,Rawson,0,AR,1585841525,35,-43.30,-65.10,19.19,1.80
7,Hermanus,0,ZA,1585841525,55,-34.42,19.23,31.67,4.02
13,Iqaluit,5,CA,1585841526,64,63.75,-68.51,-20.00,1.54
14,Pokaran,0,IN,1585841255,16,26.92,71.92,30.77,2.85
...,...,...,...,...,...,...,...,...,...
531,Hohhot,0,CN,1585842117,26,40.81,111.65,1.00,2.00
536,Westport,1,US,1585841864,46,41.14,-73.36,10.56,7.70
537,Kitangari,3,TZ,1585842143,74,-10.65,39.33,24.41,2.24
539,Pacific Grove,1,US,1585842143,70,36.62,-121.92,11.11,2.47


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [222]:
hotel_df=vacation_df[["City",'Lat','Lng','Country']]
hotel_df.reset_index(inplace=True,drop=True)

hotel_df["Hotel Name"]=""
hotel_df

/Users/rahulsanghera/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,City,Lat,Lng,Country,Hotel Name
0,Lebu,-37.62,-73.65,CL,
1,Rawson,-43.30,-65.10,AR,
2,Hermanus,-34.42,19.23,ZA,
3,Iqaluit,63.75,-68.51,CA,
4,Pokaran,26.92,71.92,IN,
...,...,...,...,...,...
152,Hohhot,40.81,111.65,CN,
153,Westport,41.14,-73.36,US,
154,Kitangari,-10.65,39.33,TZ,
155,Pacific Grove,36.62,-121.92,US,


In [225]:
index_tracker=0
target_search = "Hotel"
target_radius =5000
target_type = "Hotel"

for loc in hotel_df.iterrows():
    target_location=f"{loc[1]['Lat']}, {loc[1]['Lng']}"
    params = {
            "location":target_location,
            
            "keyword": target_search,
            "radius": target_radius,
            "type": target_type,
            "key": g_key
            }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)

    a=response.json()
    if (a["status"]=='OK'): 
        hotel_df["Hotel Name"][index_tracker]=(a["results"][0]["name"])
        index_tracker=index_tracker+1
    else:
        hotel_df["Hotel Name"][index_tracker]=("No Information Found on Google")
        index_tracker=index_tracker+1



/Users/rahulsanghera/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/rahulsanghera/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [226]:
hotel_df
        

,City,Lat,Lng,Country,Hotel Name
0,Lebu,-37.62,-73.65,CL,Hotel Plaza Lebu
1,Rawson,-43.30,-65.10,AR,Hotel Punta León
2,Hermanus,-34.42,19.23,ZA,Ocean Eleven
3,Iqaluit,63.75,-68.51,CA,Frobisher Inn
4,Pokaran,26.92,71.92,IN,The Fort Pokaran - A Heritage Hotel
...,...,...,...,...,...
152,Hohhot,40.81,111.65,CN,"Shangri-La Hotel, Hohhot"
153,Westport,41.14,-73.36,US,Hilton Garden Inn Norwalk
154,Kitangari,-10.65,39.33,TZ,No Information Found on Google
155,Pacific Grove,36.62,-121.92,US,The Monterey Hotel


In [209]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [210]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))